In [1]:
!pip install -q "openvino>=2023.1.0" onnx
!pip install opencv-python==4.8.1.78
!pip install onnxruntime==1.16.0


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import onnxruntime as rt
import numpy as np
import openvino as ov

In [3]:
#import remote_infer as ri
import remote_infer_changed_preprocessing as ri

In [4]:
image_path = 'images/carImage3.jpg' # You can replace this with an image you upload

In [5]:
preprocessed, scale, original_image = ri.preprocess(image_path)

# Serving

In [6]:
infer_url = os.environ["INFER_ENDPOINT"]

response = ri.send_request(preprocessed, infer_url)

In [7]:
response_transposed = cv2.transpose(response[0].squeeze())
print(response_transposed.shape)
print(response_transposed)

(8400, 6)
[[7.25182250e+00 1.59619200e+01 1.59490360e+01 3.18928570e+01
  2.77792900e-06 8.99698350e-06]
 [2.54508700e+01 1.32270820e+01 5.91368030e+01 2.61576290e+01
  1.19057480e-06 9.94917500e-06]
 [1.81783560e+01 1.16864980e+01 4.00377120e+01 2.24915900e+01
  3.50209300e-07 4.00328800e-06]
 ...
 [5.63258540e+02 6.07380500e+02 1.35587590e+02 6.58579100e+01
  2.88683740e-06 3.79249760e-06]
 [5.84164300e+02 5.98486000e+02 1.04063416e+02 8.50695800e+01
  2.83189700e-06 3.47373320e-06]
 [6.00892500e+02 5.86694700e+02 9.20753200e+01 1.19198730e+02
  4.63818740e-06 7.07730670e-06]]


# Session

In [8]:
ir_path = "ir_model/accident_detect.xml"
onnx_path = "accident_detect.onnx"

### IR

In [9]:
core = ov.Core()
model_ir = core.read_model(model=ir_path)

In [10]:
compiled_model_ir = core.compile_model(model=model_ir)
output_layer_ir = compiled_model_ir.output(0)
res_ir = compiled_model_ir([preprocessed])[output_layer_ir]

In [11]:
ir_sess_prediction_transposed = cv2.transpose(res_ir[0].squeeze())
print(ir_sess_prediction_transposed.shape)
print(ir_sess_prediction_transposed)

(8400, 6)
[[7.2539897e+00 1.5953136e+01 1.5954377e+01 3.1875458e+01 2.7781566e-06
  9.0011354e-06]
 [2.5465382e+01 1.3217645e+01 5.9175644e+01 2.6138700e+01 1.1910745e-06
  9.9561221e-06]
 [1.8183857e+01 1.1681372e+01 4.0054626e+01 2.2481430e+01 3.5050101e-07
  4.0072487e-06]
 ...
 [5.6326550e+02 6.0738074e+02 1.3555197e+02 6.5856018e+01 2.8873769e-06
  3.7926243e-06]
 [5.8416711e+02 5.9849036e+02 1.0404614e+02 8.5058167e+01 2.8326183e-06
  3.4742102e-06]
 [6.0090088e+02 5.8670398e+02 9.2063293e+01 1.1916858e+02 4.6395808e-06
  7.0787241e-06]]


### ONNX

In [12]:
core = ov.Core()
model_onnx = core.read_model(model=onnx_path)

In [13]:
compiled_model_onnx = core.compile_model(model=model_onnx)
res_onnx = compiled_model_onnx([preprocessed])[0]

In [14]:
onnx_sess_prediction_transposed = cv2.transpose(res_onnx[0].squeeze())
print(onnx_sess_prediction_transposed.shape)
print(onnx_sess_prediction_transposed)

(8400, 6)
[[7.25182247e+00 1.59619198e+01 1.59490356e+01 3.18928566e+01
  2.77792901e-06 8.99698352e-06]
 [2.54508705e+01 1.32270823e+01 5.91368027e+01 2.61576290e+01
  1.19057484e-06 9.94917536e-06]
 [1.81783562e+01 1.16864977e+01 4.00377121e+01 2.24915905e+01
  3.50209291e-07 4.00328781e-06]
 ...
 [5.63258545e+02 6.07380493e+02 1.35587585e+02 6.58579102e+01
  2.88683464e-06 3.79249764e-06]
 [5.84164307e+02 5.98486023e+02 1.04063416e+02 8.50695801e+01
  2.83189706e-06 3.47373657e-06]
 [6.00892517e+02 5.86694702e+02 9.20753174e+01 1.19198730e+02
  4.63818287e-06 7.07729987e-06]]


# Compare them

### IR

In [15]:
def compare_2d_arrays(arr1, arr2):
    # Ensure arr1 and arr2 have the same shape
    if arr1.shape != arr2.shape:
        raise ValueError("Arrays must have the same shape")
    
    differences = []  # List to store differences for each row
    
    # Iterate over each row
    for row_idx in range(arr1.shape[0]):
        row_differences = []  # Differences for the current row
        for col_idx in range(arr1.shape[1]):
            item1 = arr1[row_idx, col_idx]
            item2 = arr2[row_idx, col_idx]
            diff = abs(item1 - item2)  # Calculate the absolute difference
            row_differences.append((col_idx, item1, item2, diff))
        
        differences.append(row_differences)
    
    return differences

# Comparing the arrays
differences = compare_2d_arrays(response_transposed, ir_sess_prediction_transposed)

# Optionally, print the results for the first few rows to check
for row_idx, row_diff in enumerate(differences[:5]):  # Just as an example, print differences for the first 5 rows
    print(f"Row {row_idx}:")
    for diff in row_diff:
        print(f"  Col {diff[0]}: {diff[1]} (rest_prediction) vs {diff[2]} (ir_sess_prediction) - Difference:  {diff[3]}")

Row 0:
  Col 0: 7.2518225 (rest_prediction) vs 7.2539896965026855 (ir_sess_prediction) - Difference:  0.0021671965026852646
  Col 1: 15.96192 (rest_prediction) vs 15.953136444091797 (ir_sess_prediction) - Difference:  0.008783555908202345
  Col 2: 15.949036 (rest_prediction) vs 15.954377174377441 (ir_sess_prediction) - Difference:  0.00534117437744186
  Col 3: 31.892857 (rest_prediction) vs 31.875457763671875 (ir_sess_prediction) - Difference:  0.017399236328124346
  Col 4: 2.777929e-06 (rest_prediction) vs 2.778156613203464e-06 (ir_sess_prediction) - Difference:  2.2761320346421544e-10
  Col 5: 8.9969835e-06 (rest_prediction) vs 9.00113536772551e-06 (ir_sess_prediction) - Difference:  4.151867725509862e-09
Row 1:
  Col 0: 25.45087 (rest_prediction) vs 25.465381622314453 (ir_sess_prediction) - Difference:  0.014511622314454797
  Col 1: 13.227082 (rest_prediction) vs 13.217644691467285 (ir_sess_prediction) - Difference:  0.009437308532714184
  Col 2: 59.136803 (rest_prediction) vs 59.17

### ONNX

In [16]:
def compare_2d_arrays(arr1, arr2):
    # Ensure arr1 and arr2 have the same shape
    if arr1.shape != arr2.shape:
        raise ValueError("Arrays must have the same shape")
    
    differences = []  # List to store differences for each row
    
    # Iterate over each row
    for row_idx in range(arr1.shape[0]):
        row_differences = []  # Differences for the current row
        for col_idx in range(arr1.shape[1]):
            item1 = arr1[row_idx, col_idx]
            item2 = arr2[row_idx, col_idx]
            diff = abs(item1 - item2)  # Calculate the absolute difference
            row_differences.append((col_idx, item1, item2, diff))
        
        differences.append(row_differences)
    
    return differences

# Comparing the arrays
differences = compare_2d_arrays(response_transposed, onnx_sess_prediction_transposed)

# Optionally, print the results for the first few rows to check
for row_idx, row_diff in enumerate(differences[:5]):  # Just as an example, print differences for the first 5 rows
    print(f"Row {row_idx}:")
    for diff in row_diff:
        print(f"  Col {diff[0]}: {diff[1]} (rest_prediction) vs {diff[2]} (onnx_sess_prediction) - Difference:  {diff[3]}")


Row 0:
  Col 0: 7.2518225 (rest_prediction) vs 7.251822471618652 (onnx_sess_prediction) - Difference:  2.8381347938477575e-08
  Col 1: 15.96192 (rest_prediction) vs 15.961919784545898 (onnx_sess_prediction) - Difference:  2.154541007826083e-07
  Col 2: 15.949036 (rest_prediction) vs 15.94903564453125 (onnx_sess_prediction) - Difference:  3.554687495466169e-07
  Col 3: 31.892857 (rest_prediction) vs 31.89285659790039 (onnx_sess_prediction) - Difference:  4.0209960872061856e-07
  Col 4: 2.777929e-06 (rest_prediction) vs 2.7779290121543454e-06 (onnx_sess_prediction) - Difference:  1.2154345540145656e-14
  Col 5: 8.9969835e-06 (rest_prediction) vs 8.996983524411917e-06 (onnx_sess_prediction) - Difference:  2.4411916444474352e-14
Row 1:
  Col 0: 25.45087 (rest_prediction) vs 25.450870513916016 (onnx_sess_prediction) - Difference:  5.139160172973334e-07
  Col 1: 13.227082 (rest_prediction) vs 13.227082252502441 (onnx_sess_prediction) - Difference:  2.525024420663158e-07
  Col 2: 59.136803 (r